## CONTEXT

__EFFECTS OF PURCHASE LAYOUT IN PURCHASE BEHAVIOR AND SALES__

Based on the customer purchases data some stores were given new layout of the chips section. The main purpose of this task is to find out if the new layout has made any difference to the sales. For this, our main task is to compare the sales of stores with new layout to the stores with old layout during the trial period. 

__Key points:__
- Stores 77, 86 and 88 were given new layout.
- Trial Period: 01/02/2019 to 30/04/2019

__Main Tasks:__

Finding control stores for each trialed store based on
- Monthly overall sales revenue.
- Monthly number of customers.
- Monthly number of transactions per customer.

Comparing each potential control store to the trial store by a comparison metric.

In [19]:
#required libraries
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [20]:
#plot settings
sns.set_context('notebook') 
sns.set_style('ticks') 
colours = ['#4E79A7','#F28E2C','#E15759','#76B7B2','#59A14F', 
          '#EDC949','#AF7AA1','#FF9DA7','#9C755F','#BAB0AB']
sns.set_palette(colours) 
%matplotlib inline


In [21]:
df = pd.read_csv('QVI_data.csv')

In [22]:
df['DATE'] = pd.to_datetime(df['DATE'])
df['YEAR_MONTH'] = df['DATE'].dt.strftime('%Y%m').astype('int')

In [23]:
df.head()

,LYLTY_CARD_NBR,DATE,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRAND,LIFESTAGE,PREMIUM_CUSTOMER,YEAR_MONTH
0,1000,2018-10-17,1,1,5,Natural Chip Compny SeaSalt175g,2,6.0,175,NATURAL,YOUNG SINGLES/COUPLES,Premium,201810
1,1002,2018-09-16,1,2,58,Red Rock Deli Chikn&Garlic Aioli 150g,1,2.7,150,RRD,YOUNG SINGLES/COUPLES,Mainstream,201809
2,1003,2019-03-07,1,3,52,Grain Waves Sour Cream&Chives 210G,1,3.6,210,GRNWVES,YOUNG FAMILIES,Budget,201903
3,1003,2019-03-08,1,4,106,Natural ChipCo Hony Soy Chckn175g,1,3.0,175,NATURAL,YOUNG FAMILIES,Budget,201903
4,1004,2018-11-02,1,5,96,WW Original Stacked Chips 160g,1,1.9,160,WOOLWORTHS,OLDER SINGLES/COUPLES,Mainstream,201811


In [24]:
def monthly_metrics():
    monthly_groupby = df.groupby(['STORE_NBR','YEAR_MONTH'])
    monthly_sales = monthly_groupby['TOT_SALES'].sum()
    monthly_customers = monthly_groupby['LYLTY_CARD_NBR'].nunique()
    monthly_transactions = monthly_groupby.size()/monthly_customers
    price_per_unit = monthly_sales/monthly_customers
    chips_per_trans = monthly_groupby['PROD_QTY'].sum()/ monthly_groupby['TXN_ID'].nunique()
    metrics = pd.concat([monthly_sales,monthly_customers, monthly_transactions,price_per_unit,chips_per_trans], axis =1)
    metrics.columns = ["MONTHLY_SALES", "MONTHLY_CUSTOMERS", "AVERAGE_TRAX/CUSTOMER","PRICE_PER_UNIT", "CHIPS_PER_TRAX"]
    return metrics

In [67]:
monthly_data = monthly_metrics()
monthly_data.shape

(3169, 5)

In [53]:
monthly_data = monthly_data.reset_index()
observ_counts = monthly_data["STORE_NBR"].value_counts()
full_observ_index = observ_counts[observ_counts == 12].index
monthly_12_data = monthly_data[monthly_data["STORE_NBR"].isin(full_observ_index)]

In [54]:
pretrial=[]
for i in monthly_12_data.index:
    if(i[1]>=201902):
        if(i[1]<=201904):
            trial.append(monthly_12_data.loc[i])
        monthly_12_data.drop(i,inplace=True)
pretrial=pd.DataFrame(pretrial)

TypeError: 'int' object is not subscriptable

In [66]:
pretrial = monthly_12_data.loc[(monthly_12_data["YEAR_MONTH"] < 201902) & (monthly_12_data["YEAR_MONTH"] > 201904) ]
pretrial = monthly_12_data.set_index(['STORE_NBR','YEAR_MONTH'])
pretrial.shape

(3120, 6)

In [68]:
def calcCorr(store):
    '''
    input=store number which is to be compared
    output=dataframe with corelation coefficient values
    '''
    a=[]
    metrix=pretrial[['MONTHLY_SALES','MONTHLY_CUSTOMERS',"AVERAGE_TRAX/CUSTOMER","PRICE_PER_UNIT", "CHIPS_PER_TRAX"]]#add metrics as required e.g. ,'TXN_PER_CUST'
    for i in metrix.index:
        a.append(metrix.loc[store].corrwith(metrix.loc[i[0]]))
    df= pd.DataFrame(a)
    df.index=metrix.index
    df=df.drop_duplicates()
    df.index=[s[0] for s in df.index]
    df.index.name="STORE_NBR"
    return df

In [69]:
pd. set_option('display.max_rows', None) 
calcCorr(77)

,MONTHLY_SALES,MONTHLY_CUSTOMERS,AVERAGE_TRAX/CUSTOMER,PRICE_PER_UNIT,CHIPS_PER_TRAX
STORE_NBR,,,,,
1,0.064391,0.243356,0.130320,-0.457288,-0.222524
2,0.261602,0.221240,-0.260802,-0.612292,0.132556
3,0.162686,0.332938,0.173543,0.102685,0.169886
4,-0.294778,-0.384611,0.316577,0.316187,0.159010
5,-0.284544,-0.309319,0.372083,0.496659,0.239223
6,-0.021268,0.170616,0.165236,0.026346,0.058401
7,-0.158965,-0.132900,-0.687093,-0.227434,0.594987
8,-0.093946,0.113442,0.363197,0.549498,0.120874
9,-0.329342,-0.444615,0.271683,-0.253636,-0.451574


In [70]:
def standardizer(df):
    '''
    input=dataframe with metrics
    output=dataframe with mean of the metrics in a new column
    '''
    df=df.abs()
    df['MAGNITUDE']=df.mean(axis=1)
    return df

## STORE 77

In [71]:
standardizer(calcCorr(77)).sort_values('MAGNITUDE',ascending=False).dropna()

,MONTHLY_SALES,MONTHLY_CUSTOMERS,AVERAGE_TRAX/CUSTOMER,PRICE_PER_UNIT,CHIPS_PER_TRAX,MAGNITUDE
STORE_NBR,,,,,,
77,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
233,0.613063,0.676737,0.392833,0.167221,0.439467,0.457864
35,0.699708,0.787737,0.287859,0.128538,0.282185,0.437206
184,0.645118,0.562098,0.036860,0.633367,0.269619,0.429412
167,0.696075,0.748793,0.068699,0.148996,0.424691,0.417451
164,0.167261,0.472598,0.672608,0.481683,0.225443,0.403919
91,0.297867,0.274042,0.568310,0.617777,0.247225,0.401044
53,0.526309,0.576130,0.436721,0.110537,0.337809,0.397501
244,0.285043,0.322209,0.563144,0.138491,0.631733,0.388124


In [72]:
standardizer(calcCorr(86)).sort_values('MAGNITUDE',ascending=False).dropna()

,MONTHLY_SALES,MONTHLY_CUSTOMERS,AVERAGE_TRAX/CUSTOMER,PRICE_PER_UNIT,CHIPS_PER_TRAX,MAGNITUDE
STORE_NBR,,,,,,
86,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
120,0.735504,0.665852,0.247391,0.494723,0.591851,0.547064
190,0.453531,0.467470,0.493487,0.467371,0.728673,0.522106
5,0.321841,0.622949,0.328395,0.582477,0.655200,0.502172
156,0.182620,0.345427,0.662960,0.538312,0.735309,0.492926
145,0.403289,0.261957,0.664034,0.665111,0.398386,0.478555
124,0.253339,0.445638,0.575454,0.588091,0.483443,0.469193
23,0.722721,0.786055,0.059792,0.202178,0.567208,0.467591
47,0.516535,0.539808,0.313399,0.438622,0.514770,0.464627
